# 图像爬取

## 1. 导入工具包

In [6]:
# 导入操作系统库以获取平台信息以及对目录操作的权限
import os

# 导入网络请求库以支持爬虫操作
import requests

# 导入第三方网络请求库以增强爬虫操作（比标准库内的urllib更强大）
import urllib3
urllib3.disable_warnings() # 禁止urllib3验证https安全证书的警告

# 导入进度条库的模块以令进程可视化（如果仅仅 import tqdm 就需要以 tqdm.tqdm() 的方式来调用）
from tqdm import tqdm

# 导入时间库以支持睡眠操作
import time

# 导入计算扩展库的随机数模块的随机整数生成方法以增强睡眠控制
from numpy.random import uniform

## 2. 设置HTTP请求参数

In [7]:
'''
启动开发人员工具，通过 [选项卡]NetWork —> [筛选器]ALL —> [名称]目标URL 找到headers（请求标头），将里面的内容复制过来（一般反爬有User-Agent就行，不行就把所有的请求头加进去~）
注意：每个都要改写成“key: value”（键值对）形式，可以通过Regex（正则表达式）来修改内容
'''
'''
【cookies】

将原生cookies转换为字典格式：
# 方法一
Site = "" # 目标网址
Response = requests.get(Site) # 网址的响应值
Cookies = Response.cookies.get_dict() # 响应值中cookies的转换值
# 方法二
Cookies = '' # 原生Cookie值
def convert_cookies_to_dict(Cookies): # 定义转换器
    Cookies = dict([l.split("=", 1) for l in Cookies.split("; ")])
    return Cookies
print(convert_cookies_to_dict(Cookies)) # 输出转换值
'''
'''
【headers】

Referer : 当前请求页面的来源页面的地址
User-Agent : 发起请求的用户代理软件的应用类型、操作系统、软件开发商以及版本号。其中Mozilla/5.0表示与Mozilla兼容，Gecko/geckotrail表示浏览器基于Gecko渲染引擎
X-Requested-With : 客户端请求是否为Ajax请求（Ajax请求将返回JSON数据而非HTML数据，详情见爬图函数注释）。其中Ajax请求的参数为: XMLHttpRequest
Accept : 告知（服务器）客户端可以处理的内容类型。其中单一精确的MIME类型的参数为: <MIME_type>/<MIME_subtype> ，没有指明子类的一类MIME类型的参数为: <MIME_type>/* ，任意类型的MIME类型的参数为: */* ，代表优先顺序的Quality Values（q值）的参数为: ;q= 权重
Accept-Language : 允许客户端声明它可以理解的自然语言。其中语言码或语言标签的参数为: <language> （*表示任意语言），代表优先顺序的Quality Values（q值）的参数为: ;q= 权重

关于Sec-CH-UA:
source: https://developer.mozilla.org/en-US/docs/Web/HTTP/Headers/Sec-CH-UA
相比于传统的user-agent，sec-ch-ua使得headers可以包含假冒位置和名称以防止服务器直接拒绝未知的用户代理。具体的组成字段如下：
sec-ch-ua : 浏览器信息（名称与版本）。其中Chromium的参数为: "(Not(A:Brand";v="8", "Chromium";v="98" ，Edge的参数为: " Not A;Brand";v="99", "Chromium";v="96", "Microsoft Edge";v="96"
sec-ch-ua-mobile : 是否为移动设备。其中移动端浏览器的参数为: ?0 ，非移动端浏览器的参数为: ?1
sec-ch-ua-platform : 平台架构或操作系统。其中Windows系统的参数为: "Windows" ，macOS的参数为: "macOS" ，Linux的参数为: "Linux" ，以此类推
Sec-Fetch-Site : 表示一个请求发起者的来源与目标资源来源之间的关系。其中请求方和目标服务器同站时的参数为: same-site ，同域时的参数为: same-origin ，跨站时的参数为: cross-site ，跨域时的参数为: cross-origin
Sec-Fetch-Dest : 表示请求的目的地，即如何使用获取的数据。其中脚本目标的参数为: script ，嵌入内容目标的参数为: embed ，字体目标的参数为: font ，图片目标的参数为: image ，音频目标的参数为: audio ，空值（空字符串）目标的参数为: empty
Sec-Fetch-Mode : 该请求头表明了一个请求的模式。其中跨域资源共享模式的参数为: cors ，非跨域资源共享模式的参数为: no-cors ，HTML文档导航启动模式的参数为: navigate ，同域资源请求模式的参数为: same-origin ，WebSocket连接模式的参数为: WebSocket
'''

# 使用字典格式的cookie让request处理，不能放入headers
Cookies = {
'BA_HECTOR': '8h24a024042g05alup1h3g0aq0q',
'BAIDUID': '104BD58A7C408DABABCAC9E0A1B184B4:FG=1',
'BAIDUID_BFESS': '104BD58A7C408DABABCAC9E0A1B184B4:FG=1',
'BDORZ': 'B490B5EBF6F3CD402E515D22BCDA1598',
'BDRCVFR[-pGxjrCMryR]': 'mk3SLVN4HKm',
'BDRCVFR[dG2JNJb_ajR]': 'mk3SLVN4HKm',
'BDSFRCVID': '8--OJexroG0xMovDbuOS5T78igKKHJQTDYLtOwXPsp3LGJLVgaSTEG0PtjcEHMA-2ZlgogKK02OTH6KF_2uxOjjg8UtVJeC6EG0Ptf8g0M5',
'BDSFRCVID_BFESS': '8--OJexroG0xMovDbuOS5T78igKKHJQTDYLtOwXPsp3LGJLVgaSTEG0PtjcEHMA-2ZlgogKK02OTH6KF_2uxOjjg8UtVJeC6EG0Ptf8g0M5',
'BDqhfp': '%E7%8B%97%E7%8B%97%26%26NaN-1undefined%26%2618880%26%2621',
'BIDUPSID': '06338E0BE23C6ADB52165ACEB972355B',
'H_BDCLCKID_SF': 'tJPqoKtbtDI3fP36qR3KhPt8Kpby2D62aKDs2nopBhcqEIL4QTQM5p5yQ2c7LUvtynT2KJnz3Po8MUbSj4QoDjFjXJ7RJRJbK6vwKJ5s5h5nhMJSb67JDMP0-4F8exry523ioIovQpn0MhQ3DRoWXPIqbN7P-p5Z5mAqKl0MLPbtbb0xXj_0D6bBjHujtT_s2TTKLPK8fCnBDP59MDTjhPrMypomWMT-0bFH_-5L-l5js56SbU5hW5LSQxQ3QhLDQNn7_JjOX-0bVIj6Wl_-etP3yarQhxQxtNRdXInjtpvhHR38MpbobUPUDa59LUvEJgcdot5yBbc8eIna5hjkbfJBQttjQn3hfIkj0DKLtD8bMC-RDjt35n-Wqxobbtof-KOhLTrJaDkWsx7Oy4oTj6DD5lrG0P6RHmb8ht59JROPSU7mhqb_3MvB-fnEbf7r-2TP_R6GBPQtqMbIQft20-DIeMtjBMJaJRCqWR7jWhk2hl72ybCMQlRX5q79atTMfNTJ-qcH0KQpsIJM5-DWbT8EjHCet5DJJn4j_Dv5b-0aKRcY-tT5M-Lf5eT22-usy6Qd2hcH0KLKDh6gb4PhQKuZ5qutLTb4QTbqWKJcKfb1MRjvMPnF-tKZDb-JXtr92nuDal5TtUthSDnTDMRhXfIL04nyKMnitnr9-pnLJpQrh459XP68bTkA5bjZKxtq3mkjbPbDfn02eCKuj6tWj6j0DNRabK6aKC5bL6rJabC3b5CzXU6q2bDeQN3OW4Rq3Irt2M8aQI0WjJ3oyU7k0q0vWtvJWbbvLT7johRTWqR4enjb3MonDh83Mxb4BUrCHRrzWn3O5hvvhKoO3MA-yUKmDloOW-TB5bbPLUQF5l8-sq0x0bOte-bQXH_E5bj2qRCqVIKa3f',
'H_BDCLCKID_SF_BFESS': 'tJPqoKtbtDI3fP36qR3KhPt8Kpby2D62aKDs2nopBhcqEIL4QTQM5p5yQ2c7LUvtynT2KJnz3Po8MUbSj4QoDjFjXJ7RJRJbK6vwKJ5s5h5nhMJSb67JDMP0-4F8exry523ioIovQpn0MhQ3DRoWXPIqbN7P-p5Z5mAqKl0MLPbtbb0xXj_0D6bBjHujtT_s2TTKLPK8fCnBDP59MDTjhPrMypomWMT-0bFH_-5L-l5js56SbU5hW5LSQxQ3QhLDQNn7_JjOX-0bVIj6Wl_-etP3yarQhxQxtNRdXInjtpvhHR38MpbobUPUDa59LUvEJgcdot5yBbc8eIna5hjkbfJBQttjQn3hfIkj0DKLtD8bMC-RDjt35n-Wqxobbtof-KOhLTrJaDkWsx7Oy4oTj6DD5lrG0P6RHmb8ht59JROPSU7mhqb_3MvB-fnEbf7r-2TP_R6GBPQtqMbIQft20-DIeMtjBMJaJRCqWR7jWhk2hl72ybCMQlRX5q79atTMfNTJ-qcH0KQpsIJM5-DWbT8EjHCet5DJJn4j_Dv5b-0aKRcY-tT5M-Lf5eT22-usy6Qd2hcH0KLKDh6gb4PhQKuZ5qutLTb4QTbqWKJcKfb1MRjvMPnF-tKZDb-JXtr92nuDal5TtUthSDnTDMRhXfIL04nyKMnitnr9-pnLJpQrh459XP68bTkA5bjZKxtq3mkjbPbDfn02eCKuj6tWj6j0DNRabK6aKC5bL6rJabC3b5CzXU6q2bDeQN3OW4Rq3Irt2M8aQI0WjJ3oyU7k0q0vWtvJWbbvLT7johRTWqR4enjb3MonDh83Mxb4BUrCHRrzWn3O5hvvhKoO3MA-yUKmDloOW-TB5bbPLUQF5l8-sq0x0bOte-bQXH_E5bj2qRCqVIKa3f',
'H_PS_PSSID': '35836_35105_31254_36024_36005_34584_36142_36120_36032_35993_35984_35319_26350_35723_22160_36061',
'PSINO': '2',
'PSTM': '1646905430',
'ab_sr': '1.0.1_Y2YxZDkwMWZkMmY2MzA4MGU0OTNhMzVlNTcwMmM2MWE4YWU4OTc1ZjZmZDM2N2RjYmVkMzFiY2NjNWM4Nzk4NzBlZTliYWU0ZTAyODkzNDA3YzNiMTVjMTllMzQ0MGJlZjAwYzk5MDdjNWM0MzJmMDdhOWNhYTZhMjIwODc5MDMxN2QyMmE1YTFmN2QyY2M1M2VmZDkzMjMyOThiYmNhZA==',
'cleanHistoryStatus': '0',
'delPer': '0',
'indexPageSugList': '%5B%22%E7%8B%97%E7%8B%97%22%5D',
}

# 设定在请求过程中传入的头部信息以使浏览器分辨身份
Headers = {
'Referer': 'https://image.baidu.com/',
'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36 Edg/107.0.1418.52',
'X-Requested-With': 'XMLHttpRequest',
'Accept': 'text/plain, */*;q=0.01',
'Accept-Language': 'zh-CN,zh;q=0.9, en;q=0.8, en-GB;q=0.7, en-US;q=0.6',
'Connection': 'keep-alive',
'sec-ch-ua': '" Not A;Brand";v="99", "Chromium";v="96", "Microsoft Edge";v="96"',
'sec-ch-ua-mobile': '?0',
'sec-ch-ua-platform': '"Windows"',
'Sec-Fetch-Site': 'same-origin',
'Sec-Fetch-Dest': 'empty',
'Sec-Fetch-Mode': 'cors',
}

## 3. 百度爬图函数

In [8]:
'''
Ajax与JSON解析:
当我们以瀑布流的布局查看某个搜图引擎的结果页面时，通常无法一次性看到所有图片(视网站而定)。
接着，随着用户的下拉刷新或向下滚动，浏览器会监听到这一动作，并在滚动条快要达到底部时向服务器请求一个JSON数据，之后将返回的JSON数据再通过浏览器解析出来。
待解析出数据后，浏览器开始动态加载局部这个页面。注意之前加载出来的图片不会消失，而新的图片会添加到网页的尾部。
启动开发人员工具，通过 [选项卡]NetWork —> [筛选器]Fetch/XHR —> [名称]目标URL 会发现刚打开时是没有显示请求JSON数据的Request URL的，但若接着向下刷新页面，就会显示刷新时请求的Request URL了。
经多次对比发现，每次刷新Request URl只有这几个参数不同:
pn : 起始加载位置数目
rn : 每次加载总数
gsm : 十六进制形式的当前加载起始位置
word : UTF-8编码的关键词参数

百度网页采用的就是瀑布流布局加载——所有图片都是JavaScript解析出来的，也就是说根本没有把图片放在一个个HTML Tag中，因此通过XML节点解析的方法无法获取到图片的URL。
JavaScript中的真实URL一共有三个：thumbURL、middleURL、objURL：
图片质量最佳的是objURL，但采用它作为目标解析时很快会被反爬虫。
而使用thumbURL、middleURL能进行大规模的爬取，但很多图片下载下来无法打开，因此在下载时要跳过这些不正常的URL。
'''

# 定义爬取单个关键词类的函数
def Craw_SingleClass(Keyword, Download_NUM = 200): # Keyword 检索用的关键词; Download_NUM 拟爬取图像个数
    
    # 若系统路径不存在'DataSet'则新建一个'DataSet'文件夹
    if not os.path.exists('DataSet'):
        os.makedirs('DataSet')
        print('新建DataSet文件夹')

    # 若系统路径已存在'DataSet/%关键词文件夹%'则之后爬取的图片将直接保存至该文件夹
    if os.path.exists('DataSet/' + Keyword):
        print('文件夹 DataSet/{}已存在，之后直接将爬取到的图片保存至该文件夹中'.format(Keyword))
    # 否则在'DataSet'下新建一个'%关键词文件夹%'
    else:
        os.makedirs('DataSet/{}'.format(Keyword))
        print('新建文件夹：DataSet/{}'.format(Keyword))

    # 设定计数器初始值（1）
    COUNT = 1

    # 当确保进度条库在使用结束时得到了释放
    with tqdm(total = Download_NUM, position = 0, leave = True) as ProgressBar:
        
        # 设定爬取第几张（1）
        NUM = 1
        
        # 设定是否停止爬取（否）
        Break_Flag = False
        
        # 开始爬取
        while not Break_Flag:

            # 设定页数（由于百度图片每次加载30张，故不同的acjson的Request URL对应的pn值以30的量递增）
            PAGE = 30 * COUNT

            # 设定要发送的查询字符串参数（将参数放在url中传递）
            Params = (
                ('tn', 'resultjson_com'),
                ('logid', '12508239107856075440'), # ('logid', '12037596196292044001'),
                ('ipn', 'rj'),
                ('ct', '201326592'),
                ('is', ''),
                ('fp', 'result'),
                ('fr', ''),
                ('word', f'{Keyword}'),
                ('queryWord', f'{Keyword}'),
                ('cl', '2'),
                ('lm', '-1'),
                ('ie', 'utf-8'),
                ('oe', 'utf-8'),
                ('adpicid', ''),
                ('st', '-1'),
                ('z', ''),
                ('ic', ''), # ('ic', '0'),
                ('hd', ''),
                ('latest', ''),
                ('copyright', ''),
                ('s', ''),
                ('se', ''),
                ('tab', ''),
                ('width', ''),
                ('height', ''),
                ('face', '0'),
                ('istype', '2'),
                ('qc', ''),
                ('nc', '1'),
                ('expermode', ''),
                ('nojc', ''),
                ('isAsync', ''),
                ('pn', f'{PAGE}'),
                ('rn', '30'),
                ('gsm', '1e'), # ('gsm', '3c0000000000003c'),
                ('1647838001666', ''), # ('1666085342167', ''),
            )
            
            # 通过requests包向百度图片搜索引擎发送查询字符串（其中acjson为破解百度防爬虫机制的方法）
            Response = requests.get('https://image.baidu.com/search/acjson', headers = Headers, params = Params, cookies = Cookies)
            
            # 若向指定URI发出请求时返回响应对象的指示状态为正常(200为正常，404为未找到)
            if Response.status_code == 200:

                # 尝试获取返回结果为json格式的链接数据（调用非json格式的结果会返回错误）
                try:
                    JSON_Data = Response.json().get('data')

                    # 若获取成功
                    if JSON_Data:

                        # 对JSON数据文件进行遍历
                        for x in JSON_Data:

                            # 解析
                            Type = x.get('type') # 获取图像类型
                            if Type not in ['gif']: # 筛选出非gif文件
                                IMG = x.get('thumbURL') # 如果改为"objURL"（高清原图）很快会被反爬，慎用！
                                PageTitle = x.get('fromPageTitleEnc') # 图片搜索结果对应的标题（即原标题）

                                # 尝试移除SSL认证
                                try:
                                    Response = requests.get(url = IMG, verify = False)

                                    # 睡眠以控制爬虫节奏（防止IP被ban）
                                    time.sleep(uniform(1.2, 2.1))

                                    # 保存文件
                                    File_Save_Path = f'DataSet/{Keyword}/{NUM}.{Type}' # 若想保留图片原标题，在.{Type}前加上{PageTitle}即可
                                    with open(File_Save_Path, 'wb') as File: # 以“二进制·写”模式打开文件
                                        File.write(Response.content) # 将HTTP响应内容以二进制(bytes)形式写入
                                        File.flush() # 在关闭文件前刷新缓冲区
                                        NUM += 1
                                        ProgressBar.update(1) # 更新进度条

                                    # 停止爬取
                                    if NUM > Download_NUM: # 若爬取数量已达到要求
                                        if not Break_Flag:
                                            Break_Flag = True
                                        print(f'{NUM - 1}张图像爬取完毕')
                                        break
                                    
                                # 其它异常直接跳过
                                except Exception:
                                    pass
                                
                # 若返回错误直接跳过链接
                except:
                    pass
            
            # 若请求失败终止爬取
            else:
                break

            # 计数器自增（1）
            COUNT += 1

## 4. 关键词输入

- ### 爬取单类

In [ ]:
# 爬取单个关键词类的检索结果，并设定拟爬取图像个数
Craw_SingleClass('%关键词%', Download_NUM = 200)

- ### 爬取多类

In [9]:
# 设定拟爬取的多个关键词类
Class_List = ['%关键词1%', '%关键词2%', '%关键词3%', '%关键词4%', '%关键词5%', '%关键词6%', '%关键词7%', '%关键词8%', '%关键词9%']
# 对每个关键词类进行爬取，并设定拟爬取图像个数
for Each in Class_List:
    Craw_SingleClass(Each, Download_NUM = 200)